https://numba.readthedocs.io/en/stable/cuda/cooperative_groups.html

In [1]:
from numba import cuda, int32

import numpy as np


sig = (int32[:,::1],)


@cuda.jit(sig)

def sequential_rows(M):

    col = cuda.grid(1)

    g = cuda.cg.this_grid()


    rows = M.shape[0]

    cols = M.shape[1]


    for row in range(1, rows):

        opposite = cols - col - 1

        # Each row's elements are one greater than the previous row

        M[row, col] = M[row - 1, opposite] + 1

        # Wait until all threads have written their column element,

        # and that the write is visible to all other threads

        g.sync()

In [2]:
# Empty input data

A = np.zeros((1024, 1024), dtype=np.int32)

# A somewhat arbitrary choice (one warp), but generally smaller block sizes

# allow more blocks to be launched (noting that other limitations on

# occupancy apply such as shared memory size)

blockdim = 32

griddim = A.shape[1] // blockdim

In [4]:
# Kernel launch - this is implicitly a cooperative launch

sequential_rows[griddim, blockdim](A)


# What do the results look like?

# print(A)

#

# [[   0    0    0 ...    0    0    0]

#  [   1    1    1 ...    1    1    1]

#  [   2    2    2 ...    2    2    2]

#  ...

#  [1021 1021 1021 ... 1021 1021 1021]

#  [1022 1022 1022 ... 1022 1022 1022]

#  [1023 1023 1023 ... 1023 1023 1023]]

print(A)

[[   0    0    0 ...    0    0    0]
 [   1    1    1 ...    1    1    1]
 [   2    2    2 ...    2    2    2]
 ...
 [1021 1021 1021 ... 1021 1021 1021]
 [1022 1022 1022 ... 1022 1022 1022]
 [1023 1023 1023 ... 1023 1023 1023]]


/home/thiago.esterci/.conda/envs/CuNumba/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [5]:
overload = sequential_rows.overloads[(int32[:,::1],)]
max_blocks = overload.max_cooperative_grid_blocks(blockdim)
print(max_blocks)
# 1152 (e.g. on Quadro RTX 8000 with Numba 0.52.1 and CUDA 11.0)

416
